<a href="https://colab.research.google.com/github/jrhumberto/cd/blob/main/004_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fontes:
- https://medium.com/fintechexplained/using-python-to-summarize-text-articles-7f1b248d9b43

In [1]:
!pip install spacy

In [2]:
import sys
!{sys.executable} -m spacy download en

     |████████████████████████████████| 12.0 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [13]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from heapq import nlargest
punctuations = string.punctuation
from spacy.language import Language
nlp = English()
#nlp.add_pipe('sentencizer') # updated
nlp.add_pipe(nlp.create_pipe('sentencizer'))

#nlp = spacy.blank("en")

#nlp.add_pipe("sentencizer")
parser = English()

In [14]:
def pre_process(document):
    clean_tokens = [ token.lemma_.lower().strip() for token in document ]
    clean_tokens = [ token for token in clean_tokens if token not in STOP_WORDS and token not in punctuations ]
    tokens = [token.text for token in document]
    lower_case_tokens = list(map(str.lower, tokens))
    
    return lower_case_tokens

In [15]:
def generate_numbers_vector(tokens):
    frequency = [tokens.count(token) for token in tokens]
    token_dict = dict(list(zip(tokens,frequency)))
    maximum_frequency=sorted(token_dict.values())[-1]
    normalised_dict = {token_key:token_dict[token_key]/maximum_frequency for token_key in token_dict.keys()}
    return normalised_dict

In [16]:
def sentences_importance(text, normalised_dict):
    importance ={}
    for sentence in nlp(text).sents:
        for token in sentence:
            target_token = token.text.lower()
            if target_token in normalised_dict.keys():
                if sentence in importance.keys():
                    importance[sentence]+=normalised_dict[target_token]
                else:
                    importance[sentence]=normalised_dict[target_token]
    return importance

In [17]:
def generate_summary(rank, text):
    target_document = parser(text)
    importance = sentences_importance(text, generate_numbers_vector(pre_process(target_document)))
    summary = nlargest(rank, importance, key=importance.get)
    return summary

In [19]:
text = '''
When you call nlp on a text, spaCy will tokenize it and then call each component on the Doc, in order. 
Since the model data is loaded, the components can access it to assign annotations to the Doc object, 
and subsequently to the Token and Span which are only views of the Doc, and don’t own any data themselves. 
All components return the modified document, which is then processed by the next component in the pipeline.
'''
num_sentences_to_generate = 3
generate_summary(num_sentences_to_generate, text)

[
 Since the model data is loaded, the components can access it to assign annotations to the Doc object, 
 and subsequently to the Token and Span which are only views of the Doc, and don’t own any data themselves.,
 
 When you call nlp on a text, spaCy will tokenize it and then call each component on the Doc, in order.,
 
 All components return the modified document, which is then processed by the next component in the pipeline.]